In [1]:
import networkx as nx
import matplotlib.pyplot as plt

1. Create Graph

In [2]:
def read_edges(file_path):
    edges = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            node1, node2 = map(int, line.strip().split('\t'))
            edges.append((node1, node2))
    return edges
edge_file_path = 'erdos_edges.txt'
edges = read_edges(edge_file_path)
# Create graph
G = nx.Graph()
G.add_edges_from(edges)

In [5]:
import numpy as np
import scipy.sparse as sp

def sparse_to_tuple(sparse_mx):
    """Convert sparse matrix to tuple representation."""
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape

def mask_test_edges(adj):
    adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
    adj.eliminate_zeros()
    # Check that diag is zero:
    assert np.diag(adj.todense()).sum() == 0

    adj_triu = sp.triu(adj)
    adj_tuple = sparse_to_tuple(adj_triu)
    edges = adj_tuple[0]
    edges_all = sparse_to_tuple(adj)[0]
    num_test = int(np.floor(edges.shape[0] / 10.))
    num_val = int(np.floor(edges.shape[0] / 20.))

    all_edge_idx = list(range(edges.shape[0]))
    np.random.shuffle(all_edge_idx)
    val_edge_idx = all_edge_idx[:num_val]
    test_edge_idx = all_edge_idx[num_val:(num_val + num_test)]
    test_edges = edges[test_edge_idx]
    val_edges = edges[val_edge_idx]
    train_edges_positive = np.delete(edges, np.hstack([test_edge_idx, val_edge_idx]), axis=0)

    def ismember(a, b, tol=5):
        rows_close = np.all(np.round(a - b[:, None], tol) == 0, axis=-1)
        return np.any(rows_close)

    test_edges_false = []
    while len(test_edges_false) < len(test_edges):
        idx_i = np.random.randint(0, adj.shape[0])
        idx_j = np.random.randint(0, adj.shape[0])
        if idx_i == idx_j:
            continue
        if ismember([idx_i, idx_j], edges_all):
            continue
        if test_edges_false:
            if ismember([idx_j, idx_i], np.array(test_edges_false)):
                continue
            if ismember([idx_i, idx_j], np.array(test_edges_false)):
                continue
        test_edges_false.append([idx_i, idx_j])

    val_edges_false = []
    while len(val_edges_false) < len(val_edges):
        idx_i = np.random.randint(0, adj.shape[0])
        idx_j = np.random.randint(0, adj.shape[0])
        if idx_i == idx_j:
            continue
        if ismember([idx_i, idx_j], train_edges_positive):
            continue
        if ismember([idx_j, idx_i], train_edges_positive):
            continue
        if ismember([idx_i, idx_j], val_edges):
            continue
        if ismember([idx_j, idx_i], val_edges):
            continue
        if val_edges_false:
            if ismember([idx_j, idx_i], np.array(val_edges_false)):
                continue
            if ismember([idx_i, idx_j], np.array(val_edges_false)):
                continue
        val_edges_false.append([idx_i, idx_j])

    train_edges_negative = []
    while len(train_edges_negative) < len(train_edges_positive):
        idx_i = np.random.randint(0, adj.shape[0])
        idx_j = np.random.randint(0, adj.shape[0])
        if idx_i == idx_j:
            continue
        if ismember([idx_i, idx_j], edges_all):
            continue
        if ismember([idx_j, idx_i], edges_all):
            continue
        if train_edges_negative:
            if ismember([idx_j, idx_i], np.array(train_edges_negative)):
                continue
            if ismember([idx_i, idx_j], np.array(train_edges_negative)):
                continue
        train_edges_negative.append([idx_i, idx_j])

    assert not ismember(test_edges_false, edges_all)
    assert not ismember(val_edges_false, edges_all)
    assert not ismember(val_edges, train_edges_positive)
    assert not ismember(test_edges, train_edges_positive)
    assert not ismember(val_edges, test_edges)

    data = np.ones(train_edges_positive.shape[0])

    # Re-build adj matrix
    adj_train = sp.csr_matrix((data, (train_edges_positive[:, 0], train_edges_positive[:, 1])), shape=adj.shape)
    adj_train = adj_train + adj_train.T

    train_edges_negative = np.array(train_edges_negative)

    return adj_train, train_edges_positive, train_edges_negative, val_edges, val_edges_false, test_edges, test_edges_false



In [6]:
import networkx as nx
np.random.seed(0) # make sure train-test split is consistent between notebooks
adj_sparse = nx.to_scipy_sparse_array(G)

# Perform train-test split
adj_train, train_edges, train_edges_false, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges(adj_sparse)
g_train = nx.from_scipy_sparse_array(adj_train) # new graph object with only non-hidden edges

AssertionError: 

In [ ]:
print ("Total nodes:", adj_sparse.shape[0])
print ("Total edges:", int(adj_sparse.nnz/2)) # adj is symmetric, so nnz (num non-zero) = 2*num_edges
print ("Training edges (positive):", len(train_edges))
print ("Training edges (positive):", len(train_edges_false))
print ("Validation edges (positive):", len(val_edges))
print ("Validation edges (negative):", len(val_edges_false))
print ("Test edges (positive):", len(test_edges))
print ("Test edges (negative):", len(test_edges_false))

Total nodes: 13113
Total edges: 20697
Training edges (positive): 17594
Training edges (positive): 17594
Validation edges (positive): 1034
Validation edges (negative): 1034
Test edges (positive): 2069
Test edges (negative): 2069


In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
def get_roc_score(edges_pos, edges_neg, score_matrix):
    # Store positive edge predictions, actual values
    preds_pos = []
    pos = []
    for edge in edges_pos:
        preds_pos.append(score_matrix[edge[0], edge[1]]) # predicted score
        pos.append(adj_sparse[edge[0], edge[1]]) # actual value (1 for positive)
        
    # Store negative edge predictions, actual values
    preds_neg = []
    neg = []
    for edge in edges_neg:
        preds_neg.append(score_matrix[edge[0], edge[1]]) # predicted score
        neg.append(adj_sparse[edge[0], edge[1]]) # actual value (0 for negative)
        
    # Calculate scores
    preds_all = np.hstack([preds_pos, preds_neg])
    labels_all = np.hstack([np.ones(len(preds_pos)), np.zeros(len(preds_neg))])
    roc_score = roc_auc_score(labels_all, preds_all)
    ap_score = average_precision_score(labels_all, preds_all)
    return roc_score, ap_score

In [ ]:
# Function to create an adjacency matrix from edges
def create_adj_matrix(edges, num_nodes):
    row = np.array([edge[0] for edge in edges])
    col = np.array([edge[1] for edge in edges])
    data = np.ones(len(edges))

    adj_matrix = sp.csr_matrix((data, (row, col)), shape=(num_nodes, num_nodes))
    adj_matrix = adj_matrix + adj_matrix.T - sp.diags(adj_matrix.diagonal())
    return adj_matrix
# Determine the number of nodes
num_nodes = max(max(edges)) + 1

# Create adjacency matrix
adj_matrix = create_adj_matrix(edges, num_nodes)

In [ ]:
# Compute Jaccard Coefficients from g_train
jc_matrix = np.zeros(adj_matrix.shape)
for u, v, p in nx.jaccard_coefficient(g_train): # (u, v) = node indices, p = Jaccard coefficient
    jc_matrix[u][v] = p
    jc_matrix[v][u] = p # make sure it's symmetric
    
jc_matrix = jc_matrix / jc_matrix.max()
jc_roc, jc_ap = get_roc_score(test_edges, test_edges_false, jc_matrix)

print('Jaccard Coefficient Test ROC score: ', str(jc_roc))
print('Jaccard Coefficient Test AP score: ', str(jc_ap))

Jaccard Coefficient Test ROC score:  0.6587811139187635
Jaccard Coefficient Test AP score:  0.6307656162532899


In [ ]:
aa_matrix = np.zeros(adj_matrix.shape)
for u, v, p in nx.adamic_adar_index(g_train): 
    aa_matrix[u][v] = p
    aa_matrix[v][u] = p 
    
aa_matrix = aa_matrix / aa_matrix.max()

aa_roc, aa_ap = get_roc_score(test_edges, test_edges_false, aa_matrix)

print('Adamic-Adar Test ROC score: ', str(aa_roc))
print('Adamic-Adar Test AP score: ', str(aa_ap))

Adamic-Adar Test ROC score:  0.6616682174034009
Adamic-Adar Test AP score:  0.6640335868490647


In [ ]:
pa_matrix = np.zeros(adj_matrix.shape)
for u, v, p in nx.preferential_attachment(g_train): # (u, v) = node indices, p = Jaccard coefficient
    pa_matrix[u][v] = p
    pa_matrix[v][u] = p # make sure it's symmetric
    

pa_matrix = pa_matrix / pa_matrix.max()
pa_roc, pa_ap = get_roc_score(test_edges, test_edges_false, pa_matrix)

print('Preferential Attachment Test ROC score: ', str(pa_roc))
print('Preferential Attachment Test AP score: ', str(pa_ap))

Preferential Attachment Test ROC score:  0.5603285256990521
Preferential Attachment Test AP score:  0.7437701543970968
